In [45]:
#处理标签 将其处理成one-hot向量形式
def labelTrans(label):
    for i in range(len(label)):
        if label[i] =="1":
            label[i] = 0
        if label[i] =="0":
            label[i] = 1
        if label[i]=="-1":
            label[i]=2
    batch_size = 128
    class_num=3
    label = np.array(label)
    label = torch.from_numpy(label)
    label = label.view(128,1)
    one_hot = torch.zeros(batch_size, class_num).scatter_(1, label, 1)
    return one_hot

In [46]:
#处理输入进的文本 分词 转换成索引 处理成等长90的序列
import pickle
def wordProcess(text):
    weibo_index=[]#转换为每一个元素都是一个句子中单词的序号表示
    weibo_temp=[]
    weibo_processed=[]
    weibo=[]#记录每个句子分词且处理完停用词后的状态
    weibo_out=[]
    for each in text:
        weibo_processed.append(jieba.cut(each))
    
    stopwords = [line.strip() for line in open('stop_words.txt',encoding='UTF-8').readlines()]
    for row in weibo_processed:
        for word in row:
            if word not in stopwords:
                weibo_out.append(word)
        weibo.append(weibo_out)
        weibo_out=[]    
    
    vocab_to_int={}
    with open("vocab_to_int.file", "rb") as f:
        vocab_to_int = pickle.load(f)
    
    for each in weibo:
        for word in each:
            if word in vocab_to_int:
                weibo_temp.append(vocab_to_int[word])
            else:
                word = 0
        weibo_index.append(weibo_temp)
        weibo_temp=[]
    #处理成等长序列 综合来看 处理成90效果更好
    for i in range(len(weibo_index)):
        if len(weibo_index[i])>110:
            while (len(weibo_index[i])-110>0):
                weibo_index[i].pop()
        if len(weibo_index[i])==110:
            weibo_index[i]=weibo_index[i]
        else:
            m=110-len(weibo_index[i])
            for j in range(m):
                weibo_index[i].append(2)
    for i in range(len(weibo_index)):
        weibo_index[i] = list(map(int,weibo_index[i]))
    weibo_index=  torch.LongTensor(weibo_index)
    return weibo_index

In [47]:
#定义TimeDistributed层 将 (seq_len,batch_size,x)----(seq_len,batch_size,y)
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first
    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        # reshape input data --> (samples * timesteps, input_size)
        # squash timesteps
        x_reshaped = x.contiguous().view(-1, x.size(-1))
        y = self.module(x_reshaped)
        # We have to reshape Y
        if self.batch_first:
            # (samples, timesteps, output_size)
            y = y.contiguous().view(x.size(0), -1, y.size(-1))
        else:
            # (timesteps, samples, output_size)
            y = y.contiguous().view(-1, x.size(1), y.size(-1))
        return y

In [48]:
#定义Vgg类 抽取掉最后一层全联接层 并下载其余层的权重参数
class Vgg(nn.Module):
    def __init__(self):
        super(Vgg, self).__init__()
        VGG = models.vgg19(pretrained=False)
        path="/Users/xiaoqiulu/Desktop/vgg19-dcbb9e9d.pth"
        pre=torch.load(path)
        VGG.load_state_dict(pre)
        VGG.eval()
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-1])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
 
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output




In [49]:
import re
def tokenize(text):

    try:
        text = text.decode('utf-8').lower()
    except:
        text = text.encode('utf-8').decode('utf-8').lower()
    text = re.sub(u"\u2019|\u2018", "\'", text)
    text = re.sub(u"\u201c|\u201d", "\"", text)
    text = re.sub(r"http[s]?:[^\ ]+", " ", text)
    text = re.sub(r"&gt;", " ", text)
    text = re.sub(r"&lt;", " ", text)
    text = re.sub(r"&quot;", " ", text)
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"\"", " ", text)
    text = re.sub(r"#\ ", "#", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\\", " ", text)
    text = re.sub(r"[\(\)\[\]\{\}]", r" ", text)
    text = re.sub(r"#", " #", text)
    text = re.sub(r"\@", " \@", text)
    text = re.sub(r"[\!\?\.\,\+\-\$\%\^\>\<\=\:\;\*\(\)\{\}\[\]\/\~\&\'\|]", " ", text)

    words = text.split()
    return words


In [50]:
import re
def clean_space(text):
    """"
    处理多余的空格
    """
    match_regex = re.compile(u'[\u4e00-\u9fa5。\.,，:：《》、\(\)（）]{1} +(?<![a-zA-Z])|\d+ +| +\d+|[a-z A-Z]+')
    should_replace_list = match_regex.findall(text)
    order_replace_list = sorted(should_replace_list,key=lambda i:len(i),reverse=True)
    for i in order_replace_list:
        if i == u' ':
            continue
        new_i = i.strip()
        text = text.replace(i,new_i)
    return text

In [467]:
#写入TextImgdata 测试集数据 包含了文本 图像1路径 图像2路径 标签
import csv
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import re
def clean_space(text):
    """"
    处理多余的空格
    """
    match_regex = re.compile(u'[\u4e00-\u9fa5。\.,，:：《》、\(\)（）]{1} +(?<![a-zA-Z])|\d+ +| +\d+|[a-z A-Z]+')
    should_replace_list = match_regex.findall(text)
    order_replace_list = sorted(should_replace_list,key=lambda i:len(i),reverse=True)
    for i in order_replace_list:
        if i == u' ':
            continue
        new_i = i.strip()
        text = text.replace(i,new_i)
    return text
np.set_printoptions(suppress=True)
file = open("TextImgData.txt","w")    #格式 文本 图像1路径 图像2路径 标签
with open('/Users/xiaoqiulu/Desktop/nCoV_100k_train.labled.csv',"r",encoding="gb18030",errors='ignore') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        content = row[r"微博中文内容"]
        id=row[r"微博id"]
        img_path1="/Users/xiaoqiulu/Desktop/nCoV_100k_train.labled/"+id+"_1.jpg"
        img_path1=img_path1.replace(' ','')
        img_path2="/Users/xiaoqiulu/Desktop/nCoV_100k_train.labled/"+id+"_2.jpg"
        img_path2=img_path2.replace(' ','')
        if content=="":
            content = "肺炎"
        else:
            clean_space(content)
            content = content.replace("\u3000","")
        file.write(content+" "
                       +img_path1+" "
                       +img_path2+" "
                       +row[r"情感倾向"]+"\n")
       

In [460]:
#检查训练集中的标签 
with open('/Users/xiaoqiulu/Desktop/nCoV_100k_train.labled.csv',"r",encoding="gb18030",errors='ignore') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        content = row[r"微博中文内容"]
        id=row[r"微博id"]
        label = row[r"情感倾向"]
        if (label !="1")&(label!="-1")&(label!="0"):
            print(id)
        
       

In [27]:
#写入vocab_to_int文件
import pickle
with open("vocab_to_int.file", "wb") as f:
    pickle.dump(vocab_to_int, f)

478602


In [ ]:
#读取vocab_to_int文件
d = {}
import pickle
with open("vocab_to_int.file", "rb") as f:
    d = pickle.load(f)

In [20]:
#制作vocab_to_int 将每一个词汇转换成索引
import jieba
from collections import Counter
import torchvision.models as models
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import itertools
import csv
from gensim.models import Word2Vec
from torch.utils.data import Dataset
from collections import Counter
#将单词转换成索引
"""
with open("nCoV_100k_train.labled.csv","r",encoding="gb18030",errors='ignore') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        weibo.append(row[u"微博中文内容"])
"""

"""
weibo_processed=[]
weibo_data=open("weibo.txt").readlines()
for row in weibo_data:
    weibo_processed.append(list(jieba.cut(row)))
     
stopwords = [line.strip() for line in open('stop_words.txt',encoding='UTF-8').readlines()]
weibo_out=[]
weibo = []#每个元素是一个句子的分词
for row in weibo_processed:
    for word in row:
        if word not in stopwords:
            weibo_out.append(word)
    weibo.append(weibo_out)
    weibo_out=[]
#embed_dim=32
#model = Word2Vec(list(weibo), min_count=1, size=embed_dim)

"""
stopwords = [line.strip() for line in open('stop_words.txt',encoding='UTF-8').readlines()]
with open('weibo_all_text.txt', 'r') as f:
    content = f.read()
words = list(jieba.cut(content))#没有处理过的单词流
words_stream=[]#分词之后经由stopwords处理成单词流
for word in words:
    if word not in stopwords:
        words_stream.append(word)
counts = Counter(words_stream)#快速计数
vocab = sorted(counts, key=counts.get, reverse=True)#降序排列
#将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
"""
weibo_index=[]#转换为每一个元素都是一个句子中单词的序号表示
weibo_temp=[]
for each in weibo:
    for word in each:
        if word in vocab_to_int:
            weibo_temp.append(vocab_to_int[word])
        else:
            word = 0
    weibo_index.append(weibo_temp)
    weibo_temp=[]
weibo_index_pad = list(itertools.zip_longest(*weibo_index,fillvalue=2))#补全每个句子的单词数量
weibo_input = torch.LongTensor(weibo_index_pad)#词向量层的输入
"""


'\nweibo_index=[]#转换为每一个元素都是一个句子中单词的序号表示\nweibo_temp=[]\nfor each in weibo:\n    for word in each:\n        if word in vocab_to_int:\n            weibo_temp.append(vocab_to_int[word])\n        else:\n            word = 0\n    weibo_index.append(weibo_temp)\n    weibo_temp=[]\nweibo_index_pad = list(itertools.zip_longest(*weibo_index,fillvalue=2))#补全每个句子的单词数量\nweibo_input = torch.LongTensor(weibo_index_pad)#词向量层的输入\n'

In [51]:
import jieba
from collections import Counter
import torchvision.models as models
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import itertools
import csv
from torch.utils.data import Dataset
import numpy as np
import os
from torch.autograd import Variable

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_size = 32
embed_size = 32
num_layers = 2
output_size = 32
num_epochs = 20
learning_rate=1e-5
batch_size = 128
embedding_matrix = np.load("embedding_matrix_all.npy")
Imgname=os.listdir("/Users/xiaoqiulu/Desktop/nCoV_100k_train.labled")
class TextImgDataSet(Dataset):
    def __init__(self,txt_path):
        file = open(txt_path, 'r')
        data=[]#格式为：文本 图像1 图像2 标签
        for line in file:
            line = line.rstrip()#删除 string 字符串末尾的指定字符（默认为空格）.
            words = line.split()#按照空格分词
            data.append((words[0],words[1],words[2], words[3]))
        self.data = data 
    def __getitem__(self, index):
        text,imgpath1,imgpath2,label = self.data[index]
        if imgpath1 in Imgname:
            img1 = Image.open(imgpath1).convert('RGB')
        else:
            img1 = Image.new('RGB', (150, 150), (255, 255, 255))
        if imgpath2 in Imgname:
            img2 = Image.open(imgpath2).convert("RGB")
        else:
            img2 = Image.new('RGB', (150, 150), (255, 255, 255))
        transform=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
        img1 = transform(img1)
        img2 = transform(img2)
        return text,img1,img2,label
    def __len__(self):
        return len(self.data)
trainset = TextImgDataSet("TextImgData.txt")
trainloader =  torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=False)
class MVAE(nn.Module):
    def __init__(self,output_size,hidden_size,embed_size,num_layers,embedding_matrix,Vgg,bidirectional=True):
        super(MVAE,self).__init__()
        self.output_size = output_size
        self.hidden_size=hidden_size
        self.embed_size=embed_size
        self.embedding_matrixa=embedding_matrix
        self.bidirectional=bidirectional
        self.num_layers=num_layers
        self.vgg_model=Vgg
        #冻结VGG-19模型部分参数
        #self.vgg_model=Vgg()
        #self.vgg_model = nn.Sequential(*list(model.children())[:-1])
        #self.vgg_model = Vgg(models.vgg19(pretrained=False))
        #print(self.parameters)
        for p in self.parameters():
            p.requires_grad=False
        
        
    
        #编码器部分参数
        self.embedding= nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix))
        self.lstm=nn.LSTM(embed_size, hidden_size, num_layers,bidirectional=bidirectional)
        #self.text_fc = nn.Linear(hidden_size*2, output_size)
        self.text_fc = nn.Linear(7040, output_size)
        self.vgg_fc1=nn.Linear(4096,1024)
        self.vgg_fc2 = nn.Linear(1024,32)
        #latent-dim部分参数
        self.latent_fc1 = nn.Linear(64,48)#latent_dim
        self.latent_fc2 = nn.Linear(48,32)#mu
        self.latent_fc3 = nn.Linear(48,32)#log_var
        #识别器部分参数
        self.fc1=nn.Linear(32,64)
        self.fc2 = nn.Linear(64,32)
        self.detect_fc = nn.Linear(32,3)
        #解码器部分参数
        self.text_decode_fc = nn.Linear(32,3520)
        self.decode_lstm=nn.LSTM(embed_size, hidden_size, num_layers,bidirectional=bidirectional)
        self.timedistributed = TimeDistributed(nn.Linear(64,32))
        self.img_decode_fc1 = nn.Linear(32,1024)
        self.img_decode_fc2=nn.Linear(1024,4096)
        
     #定义文本编码器 包含Embedding层 Bi-LSTM层以及一个全联接层   
    def textEncoder(self,x):
        h0 = torch.zeros(self.num_layers*2, batch_size, self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, batch_size, self.hidden_size).to(device)
        embeds = self.embedding(x)#执行
        #print(embeds.dtype)#torch.float64
        #print(h0.dtype)#torch.float32
        embeds=embeds.to(torch.float32)
        #print(embeds.shape)#torch.Size([110, 128, 32])
        #print(embeds.dtype)
        lstm_out, _ = self.lstm(embeds, (h0,c0))#不执行110*128*64
        #print(lstm_out.shape)#torch.Size([110, 128, 64])
        #print(22)
        #out = lstm_out.contiguous().view(-1, self.hidden_size*2)
        lstm_out=lstm_out.view(128,-1)
        text_out = self.text_fc(lstm_out)
        #print(text_out.shape)#torch.Size([128, 32])
        return embeds,text_out
    
    #定义图像编码器 包含一个更新过的VGG-19模型
    def imgEncoder(self,y1,y2):
        """
        path="/Users/xiaoqiulu/Desktop/vgg19-dcbb9e9d.pth"
        pre=torch.load(path)
        self.vgg_model.load_state_dict(pre)
        self.vgg_model.eval()
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])#这是imagenet数据集的均值
        tran=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
        """
        #更改VGG模型的最后两层 去掉最后一个全联接层，增加两个全联接层
        #print(y1.shape)#torch.Size([128, 3, 150, 150])
        img_hidden_out1 = self.vgg_model(y1)#中间输出
        img_hidden_out2 = self.vgg_model(y2)#中间输出
        #print(img_hidden_out1.shape)
        img_out1 = self.vgg_fc1(img_hidden_out1)
        img_out1=self.vgg_fc2(img_out1)
        img_out2 = self.vgg_fc1(img_hidden_out2)
        img_out2 = self.vgg_fc2(img_out2)
        #print(img_out1.shape)#torch.Size([128, 32])
        return (img_hidden_out1+img_hidden_out2)/2,(img_out1+img_out2)/2
    
    #拼接函数 将文本编码和图像编码拼接
    def concatenate(self,x,y):
        out = torch.cat((x,y),-1)#64维
        return out
        #print(out.shape)#torch.Size([128, 64])
        
    #获取特征 通过一个全联接层得到均值和方差
    def featureExtracter(self,out):#得到均值与方差
        h = self.latent_fc1(out)
        #print(h.shape)
        h=F.relu(h)
        #print(h.shape)
        return self.latent_fc2(h),self.latent_fc3(h)
    
    #重参数函数 采样得到新的样本
    def reparameterize(self,mu,log_var):
        std = torch.exp(log_var/2)# 这一步是将方差的对数转换为标准差
        eps = torch.randn_like(std)# 生成一个跟std同样大小的取自标准正态分布的随机数
        return mu + eps * std# 根据标准正态分布的采样得到之前正态分布中的采样
    
    #文本解码器 
    def textDecoder(self,z):
        h0 = torch.zeros(self.num_layers*2, batch_size, self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, batch_size, self.hidden_size).to(device)
        z_fc = self.text_decode_fc(z)#执行
        #print(z_fc.shape)#torch.Size([128, 3520])
        z_fc=z_fc.view(-1,128,32)
        #print(z_fc.shape)#torch.Size([110, 128, 32])
        lstm_out, _ = self.decode_lstm(z_fc,(h0,c0))
        #print(lstm_out.shape)#torch.Size([110, 128, 64])
        lstm_out = self.timedistributed(lstm_out)
        text_reconst = F.sigmoid(lstm_out)
        #print(text_reconst.shape)#torch.Size([110, 128, 32])
        return text_reconst
        
    #图像解码器
    def imgDecoder(self,z):
        z_fc1=self.img_decode_fc1(z)
        z_fc2=self.img_decode_fc2(z_fc1)
        img_reconst=F.sigmoid(z_fc2)
        #print(img_reconst.shape)#torch.Size([128, 4096])
        return img_reconst
        
    #情绪识别器 包含两个全联接层
    def detector(self,z):
        z1=F.relu(self.fc1(z))
        z2=F.relu(self.fc2(z1))
        z_kind = self.detect_fc(z2)
        return F.sigmoid(z_kind)
    
    #前向函数
    def forward(self,x,y1,y2):
        #编码部分
        embeds,text_out = self.textEncoder(x)
        img_hidden_out,img_out = self.imgEncoder(y1,y2)
        #print(text_out.shape)#torch.Size([128, 32])
        #print(img_out.shape)#torch.Size([128, 32])
        latent_out = self.concatenate(text_out,img_out)
        #重参数部分
        mu,log_var=self.featureExtracter(latent_out)
        z = self.reparameterize(mu,log_var)
        #解码部分
        text_reconst = self.textDecoder(z)
        img_reconst = self.imgDecoder(z)
        #情绪识别部分
        z_kind = self.detector(z)
        return z_kind,mu,log_var,text_reconst,img_reconst,embeds,img_hidden_out
    

  
        

In [52]:
Vgg=Vgg()
model = MVAE(output_size,hidden_size,embed_size,num_layers,embedding_matrix,Vgg).to(device)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
   
for epoch in range(num_epochs):
    for i, (text,img1,img2,label) in enumerate(trainloader):
        #label = Variable(torch.from_numpy(np.array(label)).long())
        #label = torch.tensor(label)
        #print(len(label))#128
        #print(label[1])#-1
        labels=[]
        for each in label:
            labels.append(each)
        #print(labels)
        label = labelTrans(labels)
        #print(label)
        #print(label[0])#[0,1,0]
        #print(label[1])#[0,0,1]
        #print(label[2])#[1,0,0]
        text = wordProcess(text)
        text = text.view(110,128)
        z_kind,mu,log_var,text_reconst,img_reconst,embeds,img_hidden_out = model(text,img1,img2)
        print(z_kind)
    
        text_reconst_loss = F.binary_cross_entropy(text_reconst,embeds, size_average=True) 
        loss_fn = torch.nn.MSELoss(reduce=True, size_average=True)
        img_reconst_loss = loss_fn(img_reconst,img_hidden_out) 
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()) # 计算KL散度 
        detector_loss = F.binary_cross_entropy(z_kind,label,size_average=True)
        loss = text_reconst_loss+img_reconst_loss+kl_div+detector_loss
            
        optimizer.zero_grad() # 梯度置零
        loss.backward() # 反向传播
        optimizer.step() # 步进 更新权值参数
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Loss: {:.4f}" .format(epoch+1, num_epochs, i+1, len(trainloader),loss.item()))
    

/Users/xiaoqiulu/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/xiaoqiulu/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


tensor([[0.5651, 0.4853, 0.5099],
        [0.5398, 0.4891, 0.4958],
        [0.5479, 0.4658, 0.4918],
        [0.5489, 0.4881, 0.4814],
        [0.5537, 0.4703, 0.4812],
        [0.5967, 0.4563, 0.5084],
        [0.5658, 0.4753, 0.4938],
        [0.5814, 0.4620, 0.4937],
        [0.5749, 0.4784, 0.5219],
        [0.5449, 0.4770, 0.4931],
        [0.5689, 0.4519, 0.5258],
        [0.5741, 0.4715, 0.4853],
        [0.5433, 0.4783, 0.4852],
        [0.5722, 0.4673, 0.5109],
        [0.5719, 0.4531, 0.4818],
        [0.6051, 0.4552, 0.5142],
        [0.5509, 0.4751, 0.5039],
        [0.5710, 0.4853, 0.4812],
        [0.5607, 0.4788, 0.5103],
        [0.5656, 0.4633, 0.4768],
        [0.5490, 0.4744, 0.4774],
        [0.5805, 0.4562, 0.5266],
        [0.5457, 0.4752, 0.4858],
        [0.5972, 0.4541, 0.4949],
        [0.5606, 0.4541, 0.4947],
        [0.5823, 0.4702, 0.5049],
        [0.5625, 0.4686, 0.5041],
        [0.5671, 0.4686, 0.5081],
        [0.5582, 0.4760, 0.4852],
        [0.564

tensor([[0.6000, 0.4349, 0.5352],
        [0.5572, 0.4788, 0.5166],
        [0.5787, 0.4483, 0.5187],
        [0.5669, 0.4510, 0.5096],
        [0.5529, 0.4855, 0.5206],
        [0.5456, 0.4670, 0.4863],
        [0.5783, 0.4624, 0.4894],
        [0.5606, 0.4531, 0.4969],
        [0.5281, 0.4854, 0.5107],
        [0.5445, 0.4833, 0.4956],
        [0.5863, 0.4535, 0.5203],
        [0.5935, 0.4596, 0.4795],
        [0.5693, 0.4658, 0.4957],
        [0.5410, 0.4735, 0.4536],
        [0.5691, 0.4813, 0.4899],
        [0.5842, 0.4686, 0.4869],
        [0.5530, 0.5008, 0.4881],
        [0.5703, 0.4608, 0.5025],
        [0.5765, 0.4623, 0.5292],
        [0.6066, 0.4674, 0.4749],
        [0.6319, 0.4586, 0.5090],
        [0.5585, 0.4753, 0.4655],
        [0.5830, 0.4646, 0.5106],
        [0.6042, 0.4640, 0.5053],
        [0.5600, 0.4860, 0.4883],
        [0.5369, 0.4815, 0.5098],
        [0.5909, 0.4755, 0.4973],
        [0.5961, 0.4686, 0.5022],
        [0.5401, 0.4680, 0.4858],
        [0.570

tensor([[0.5795, 0.4689, 0.5216],
        [0.5487, 0.4690, 0.5116],
        [0.5553, 0.4753, 0.5010],
        [0.5508, 0.4743, 0.4935],
        [0.5698, 0.4778, 0.4948],
        [0.6010, 0.4569, 0.4795],
        [0.5796, 0.4773, 0.4855],
        [0.5733, 0.4737, 0.5034],
        [0.5729, 0.4604, 0.5279],
        [0.5617, 0.4694, 0.4918],
        [0.5892, 0.4516, 0.4974],
        [0.5788, 0.4741, 0.5159],
        [0.6029, 0.4621, 0.5096],
        [0.6099, 0.4682, 0.4932],
        [0.5618, 0.4791, 0.5371],
        [0.5604, 0.4751, 0.4984],
        [0.5393, 0.4690, 0.5073],
        [0.5523, 0.4831, 0.5136],
        [0.5943, 0.4510, 0.4957],
        [0.5865, 0.4687, 0.5020],
        [0.5590, 0.4549, 0.5068],
        [0.5869, 0.4566, 0.5237],
        [0.5880, 0.4652, 0.5068],
        [0.5592, 0.4666, 0.5228],
        [0.5545, 0.4611, 0.4790],
        [0.5733, 0.4833, 0.5086],
        [0.5784, 0.4685, 0.4949],
        [0.5397, 0.4704, 0.4678],
        [0.5660, 0.4718, 0.4941],
        [0.571

tensor([[0.6021, 0.4562, 0.5349],
        [0.5950, 0.4776, 0.5267],
        [0.5738, 0.4672, 0.5194],
        [0.6122, 0.4634, 0.5285],
        [0.5300, 0.4858, 0.4799],
        [0.5434, 0.4757, 0.5184],
        [0.5993, 0.4599, 0.4683],
        [0.5572, 0.4727, 0.4881],
        [0.5468, 0.4834, 0.4727],
        [0.5668, 0.4523, 0.4733],
        [0.5966, 0.4707, 0.4855],
        [0.5933, 0.4640, 0.5251],
        [0.5615, 0.4555, 0.4782],
        [0.6103, 0.4646, 0.5138],
        [0.5686, 0.4694, 0.5149],
        [0.5297, 0.4829, 0.5193],
        [0.5331, 0.4811, 0.4589],
        [0.5867, 0.4513, 0.5109],
        [0.5825, 0.4692, 0.4994],
        [0.5512, 0.4867, 0.4858],
        [0.5233, 0.5051, 0.4887],
        [0.5823, 0.4502, 0.5112],
        [0.5920, 0.4473, 0.5099],
        [0.5626, 0.4664, 0.5065],
        [0.5528, 0.4427, 0.5005],
        [0.5831, 0.4774, 0.4709],
        [0.6078, 0.4618, 0.5053],
        [0.5483, 0.4544, 0.4879],
        [0.5763, 0.4711, 0.5080],
        [0.549

...

tensor([[0.5519, 0.4960, 0.5028],
        [0.5795, 0.4480, 0.5040],
        [0.5669, 0.4701, 0.5031],
        [0.5578, 0.4625, 0.4931],
        [0.5595, 0.4789, 0.4674],
        [0.5650, 0.4681, 0.4926],
        [0.5698, 0.4768, 0.4936],
        [0.5604, 0.4662, 0.4782],
        [0.5523, 0.4797, 0.5321],
        [0.5841, 0.4581, 0.5327],
        [0.5651, 0.4837, 0.5108],
        [0.5265, 0.4910, 0.4846],
        [0.5571, 0.4717, 0.5116],
        [0.5528, 0.4793, 0.5300],
        [0.6172, 0.4489, 0.4913],
        [0.5899, 0.4653, 0.4906],
        [0.5935, 0.4449, 0.5365],
        [0.5769, 0.4522, 0.4800],
        [0.5502, 0.4716, 0.4954],
        [0.5395, 0.4939, 0.5028],
        [0.5444, 0.4645, 0.5324],
        [0.5970, 0.4845, 0.5014],
        [0.6044, 0.4868, 0.4871],
        [0.5594, 0.4550, 0.5055],
        [0.5593, 0.4674, 0.5337],
        [0.5679, 0.4592, 0.4972],
        [0.5869, 0.4666, 0.5162],
        [0.5924, 0.4742, 0.4981],
        [0.5601, 0.4881, 0.4933],
        [0.550

tensor([[0.5838, 0.4701, 0.4943],
        [0.5624, 0.4771, 0.5068],
        [0.5279, 0.4936, 0.5023],
        [0.5881, 0.4513, 0.5163],
        [0.5928, 0.4652, 0.5060],
        [0.5545, 0.4742, 0.5048],
        [0.5624, 0.4796, 0.5284],
        [0.5450, 0.4765, 0.4831],
        [0.5717, 0.4577, 0.4783],
        [0.5768, 0.4699, 0.5129],
        [0.5815, 0.4712, 0.5256],
        [0.6004, 0.4765, 0.4871],
        [0.5808, 0.4667, 0.4854],
        [0.5747, 0.4536, 0.5191],
        [0.5654, 0.4706, 0.5003],
        [0.5554, 0.4751, 0.4922],
        [0.5540, 0.4824, 0.4956],
        [0.5879, 0.4472, 0.5191],
        [0.5958, 0.4777, 0.5215],
        [0.5727, 0.4721, 0.5116],
        [0.5353, 0.4585, 0.4905],
        [0.5693, 0.5035, 0.5200],
        [0.5828, 0.4541, 0.4976],
        [0.5972, 0.4921, 0.4961],
        [0.5914, 0.4613, 0.5049],
        [0.6199, 0.4443, 0.5020],
        [0.5794, 0.4578, 0.5131],
        [0.6056, 0.4599, 0.5172],
        [0.5739, 0.4666, 0.5227],
        [0.555

Epoch[1/20], Step [30/782], Loss: 15.4592
tensor([[0.5685, 0.4776, 0.5003],
        [0.5736, 0.4774, 0.4787],
        [0.5573, 0.5154, 0.5081],
        [0.5801, 0.4567, 0.4943],
        [0.5696, 0.4643, 0.4725],
        [0.5800, 0.4585, 0.5057],
        [0.5759, 0.4564, 0.5089],
        [0.5687, 0.4748, 0.5117],
        [0.5888, 0.4690, 0.4704],
        [0.5409, 0.4610, 0.5316],
        [0.5931, 0.4802, 0.4738],
        [0.5845, 0.4515, 0.4952],
        [0.6017, 0.4539, 0.4842],
        [0.5609, 0.4713, 0.4787],
        [0.5463, 0.4940, 0.5081],
        [0.5605, 0.4823, 0.4921],
        [0.5506, 0.4775, 0.5091],
        [0.5885, 0.4686, 0.4953],
        [0.5601, 0.4546, 0.5093],
        [0.5527, 0.4688, 0.4875],
        [0.5472, 0.4797, 0.5169],
        [0.5672, 0.4641, 0.4933],
        [0.6070, 0.4519, 0.4924],
        [0.5645, 0.4719, 0.5001],
        [0.5661, 0.4818, 0.4988],
        [0.5582, 0.4672, 0.5097],
        [0.5794, 0.4810, 0.5097],
        [0.5807, 0.4557, 0.5070],
      

tensor([[0.5683, 0.4404, 0.5082],
        [0.5992, 0.4455, 0.5301],
        [0.5889, 0.4595, 0.4862],
        [0.6127, 0.4903, 0.5070],
        [0.5668, 0.4750, 0.4836],
        [0.6091, 0.4555, 0.5021],
        [0.5475, 0.4909, 0.4879],
        [0.5713, 0.4548, 0.5061],
        [0.5639, 0.4808, 0.5166],
        [0.5834, 0.4696, 0.5235],
        [0.5969, 0.4797, 0.5161],
        [0.5694, 0.4631, 0.4979],
        [0.5653, 0.4638, 0.5032],
        [0.5608, 0.4662, 0.4873],
        [0.5540, 0.4675, 0.4832],
        [0.5610, 0.4874, 0.5039],
        [0.5737, 0.4747, 0.5033],
        [0.5666, 0.4617, 0.5251],
        [0.5642, 0.4719, 0.4966],
        [0.5801, 0.4631, 0.5282],
        [0.5486, 0.4771, 0.4838],
        [0.5658, 0.4781, 0.5083],
        [0.5542, 0.4717, 0.4982],
        [0.5608, 0.4681, 0.5192],
        [0.5978, 0.4463, 0.4897],
        [0.5629, 0.4638, 0.4983],
        [0.5489, 0.4910, 0.4912],
        [0.5243, 0.5025, 0.4663],
        [0.6039, 0.4395, 0.5214],
        [0.552

KeyboardInterrupt: 

In [435]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [436]:
print(device)

cpu


In [437]:
torch.cuda.is_available()

False